In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/train_feat_BNF_h5_14oct_Bharat_160k.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,langid
0,1.120473,-1.207826,-1.102705,0.731326,-1.690868,1.067887,-0.014018,0.159593,0.247715,0.264993,...,-0.094636,-0.323671,0.844373,-0.366620,1.634246,0.441962,0.681505,-0.632136,-0.712648,spa-eur
1,0.855223,0.237345,-0.509226,0.297071,-1.554144,0.310795,0.177555,-1.708162,1.859202,2.637373,...,-1.858368,0.495619,-1.029202,1.921910,-0.779570,2.633960,0.226205,0.493565,-0.376497,spa-eur
2,2.178442,-0.152923,-0.277384,1.359762,-1.063728,0.251058,-0.175947,0.767762,0.104705,1.704032,...,1.819529,-1.252623,0.706020,-0.369342,-0.630677,1.568804,1.663335,0.067233,0.539783,spa-eur
3,0.020323,0.186472,-1.045840,1.076967,-0.881765,0.140140,0.056690,-1.387089,0.074638,1.077490,...,1.402422,1.356642,-0.030127,-1.133595,-1.550995,-1.052241,1.623444,-0.244312,0.853134,spa-eur
4,-1.672608,1.260625,-0.865485,1.902568,1.251961,1.508542,-0.085114,3.884414,-1.814981,0.504477,...,0.000128,0.103537,-0.044487,1.756438,2.346587,-0.574275,-0.874379,0.378014,-1.190706,spa-eur


In [5]:
train_lre.groupby(['langid']).size()

langid
ara-acm     4380
ara-apc     5500
ara-ary     4467
ara-arz    23779
eng-gbr     1104
eng-usg    14529
por-brz      962
qsl-pol     4258
qsl-rus    11552
spa-car    23799
spa-eur     2807
spa-lac    24662
zho-cmn    38465
zho-nan     3126
dtype: int64

In [6]:
val_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/dev_feat_BNF_h5_3661_uttid_Bharat_160k.csv')


In [7]:
#val_lre = val_lre.iloc[100:300]

In [8]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.098584,1.575492,-0.453370,-0.651831,0.285572,-0.726594,...,0.458423,-1.359055,0.640751,0.094361,-0.700234,-0.034466,-0.580875,0.004446,-0.104522,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.436526,1.444811,-0.589956,-0.255953,0.850315,-0.745601,...,1.388978,-1.977061,2.114798,0.012926,-1.669853,0.446671,-2.128618,-0.970636,0.766287,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,0.550556,1.410690,-0.617443,0.943157,0.864899,-0.206588,...,-0.336275,-2.661029,0.086109,0.951649,-0.078517,0.722322,-1.811978,-1.288542,-0.250591,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,0.785263,1.225041,-0.615307,-0.601939,0.067760,-0.303262,...,0.712126,-0.269891,-0.210374,-0.675105,-0.101579,0.432103,-0.307661,-0.430126,-1.499074,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.561980,1.057819,-0.846207,-0.226456,-0.017701,-0.419544,...,0.590097,-1.033919,-0.465676,-0.503004,-1.776944,0.901027,0.513566,0.419297,-0.321541,lre17_qgszpuyw


In [9]:
X_train = train_lre.drop("langid",axis=1)
y_train = train_lre["langid"]
#y_train_uttid = train_lre["uttid"]

In [10]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [11]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [12]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [13]:
y_train=le.transform(y_train)

In [14]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [15]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [16]:
y_val_labels = le.transform(y_val)

In [17]:
y_val_labels[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [18]:
X_train=X_train.values
X_val=X_val.values

In [19]:
nb_classes = 14

In [20]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [21]:
Y_train = indices_to_one_hot(y_train, 14)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val_labels, 14)

In [22]:
Y_val

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [23]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
Y_val.shape

(3661, 14)

In [25]:
Y_train.shape

(163390, 14)

In [26]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (163390, 500)
163390 train samples
3661 val samples


In [27]:
X_train.shape

(163390, 500)

In [28]:
y_train.shape

(163390,)

In [29]:
#X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [30]:
X_val.shape

(3661, 500)

In [31]:
y_val_labels

array([ 0,  0,  0, ..., 13, 13, 13])

In [32]:
#X_val,  y_val_labels = shuffle(X_val, y_val_labels, random_state=0)

In [33]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 14 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [34]:
X_dim, y_dim

(500, 14)

In [35]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [36]:
""" ==================== GENERATOR ======================== """

Wzh = xavier_init(size=[Z_dim + y_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)


def G(z, c):
    inputs = torch.cat([z, c], 1)
    h = nn.relu(inputs @ Wzh + bzh.repeat(inputs.size(0), 1))
    X = nn.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X



In [37]:
""" ==================== DISCRIMINATOR ======================== """

Wxh = xavier_init(size=[X_dim + y_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Why = xavier_init(size=[h_dim, 1])
bhy = Variable(torch.zeros(1), requires_grad=True)


def D(X, c):
    inputs = torch.cat([X, c], 1)
    h = nn.relu(inputs @ Wxh + bxh.repeat(inputs.size(0), 1))
    y = nn.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    return y


G_params = [Wzh, bzh, Whx, bhx]
D_params = [Wxh, bxh, Why, bhy]
params = G_params + D_params

In [38]:
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())


G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

#ones_label = Variable(torch.ones(mb_size))
#zeros_label = Variable(torch.zeros(mb_size))

ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))


In [39]:
len(X_train)

163390

In [40]:
batch_size = mb_size
# Start training
for epoch in range(5):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(z, c)
        D_real = D(X, c)
        D_fake = D(G_sample, c)

        D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        reset_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, c)
        D_fake = D(G_sample, c)

        G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        reset_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('epoch-{}; D_loss: {}; G_loss: {}'.format(epoch, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

epoch: 0
epoch-0; D_loss: [ 1.8257947]; G_loss: [ 1.07223976]
epoch-0; D_loss: [ 1.09073663]; G_loss: [ 1.86226988]
epoch-0; D_loss: [ 0.80273843]; G_loss: [ 2.65614676]
epoch-0; D_loss: [ 0.59948444]; G_loss: [ 3.31797338]
epoch-0; D_loss: [ 0.54428184]; G_loss: [ 3.85294795]
epoch-0; D_loss: [ 0.48215932]; G_loss: [ 4.26420403]
epoch-0; D_loss: [ 0.39375824]; G_loss: [ 4.58192015]
epoch-0; D_loss: [ 0.35386604]; G_loss: [ 4.80928898]


/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


epoch-0; D_loss: [ 0.28191307]; G_loss: [ 4.97565556]
epoch-0; D_loss: [ 0.30068669]; G_loss: [ 5.06055927]
epoch-0; D_loss: [ 0.28051004]; G_loss: [ 5.08834124]
epoch-0; D_loss: [ 0.33384556]; G_loss: [ 5.10204458]
epoch-0; D_loss: [ 0.31417003]; G_loss: [ 5.08178902]
epoch-0; D_loss: [ 0.29872376]; G_loss: [ 4.97199154]
epoch-0; D_loss: [ 0.32005253]; G_loss: [ 4.87059259]
epoch-0; D_loss: [ 0.26436955]; G_loss: [ 4.7585392]
epoch-0; D_loss: [ 0.24930729]; G_loss: [ 4.61812162]
epoch-0; D_loss: [ 0.21734224]; G_loss: [ 4.45831442]
epoch-0; D_loss: [ 0.20956622]; G_loss: [ 4.35101938]
epoch-0; D_loss: [ 0.19116847]; G_loss: [ 4.20104647]
epoch-0; D_loss: [ 0.17221187]; G_loss: [ 4.05748749]
epoch-0; D_loss: [ 0.15747344]; G_loss: [ 3.87294364]
epoch-0; D_loss: [ 0.14869714]; G_loss: [ 3.82702684]
epoch-0; D_loss: [ 0.13739578]; G_loss: [ 3.68620777]
epoch-0; D_loss: [ 0.13359424]; G_loss: [ 3.63333917]
epoch-0; D_loss: [ 0.11271997]; G_loss: [ 3.56688452]
epoch-0; D_loss: [ 0.10249139

epoch-0; D_loss: [ 0.35234517]; G_loss: [ 1.37280715]
epoch-0; D_loss: [ 0.37182075]; G_loss: [ 1.32701123]
epoch-0; D_loss: [ 0.36758211]; G_loss: [ 1.31848681]
epoch-0; D_loss: [ 0.38992533]; G_loss: [ 1.28386009]
epoch-0; D_loss: [ 0.40888542]; G_loss: [ 1.26343882]
epoch-0; D_loss: [ 0.40004432]; G_loss: [ 1.20149875]
epoch-0; D_loss: [ 0.54522985]; G_loss: [ 0.97393423]
epoch-0; D_loss: [ 0.57142967]; G_loss: [ 0.95650601]
epoch-0; D_loss: [ 0.61708689]; G_loss: [ 0.97233737]
epoch-0; D_loss: [ 0.60955846]; G_loss: [ 0.98122358]
epoch-0; D_loss: [ 0.58376408]; G_loss: [ 0.98708653]
epoch-0; D_loss: [ 0.61174202]; G_loss: [ 0.99995995]
epoch-0; D_loss: [ 0.58856964]; G_loss: [ 1.01178503]
epoch-0; D_loss: [ 0.55922663]; G_loss: [ 1.03615892]
epoch-0; D_loss: [ 0.54305792]; G_loss: [ 1.06243265]
epoch-0; D_loss: [ 0.5361219]; G_loss: [ 1.08391917]
epoch-0; D_loss: [ 0.5087446]; G_loss: [ 1.10706937]
epoch-0; D_loss: [ 0.48256892]; G_loss: [ 1.12084651]
epoch-0; D_loss: [ 0.46861583]

epoch-0; D_loss: [ 0.14955555]; G_loss: [ 2.57669377]
epoch-0; D_loss: [ 0.15050352]; G_loss: [ 2.53766274]
epoch-0; D_loss: [ 0.15263803]; G_loss: [ 2.46378684]
epoch-0; D_loss: [ 0.1481856]; G_loss: [ 2.4492991]
epoch-0; D_loss: [ 0.1403676]; G_loss: [ 2.39202738]
epoch-0; D_loss: [ 0.13167261]; G_loss: [ 2.3956778]
epoch-0; D_loss: [ 0.15043834]; G_loss: [ 2.38582873]
epoch-0; D_loss: [ 0.14398864]; G_loss: [ 2.40849948]
epoch-0; D_loss: [ 0.13506876]; G_loss: [ 2.41376281]
epoch-0; D_loss: [ 0.12628856]; G_loss: [ 2.45972204]
epoch-0; D_loss: [ 0.12145877]; G_loss: [ 2.48295045]
epoch-0; D_loss: [ 0.12170866]; G_loss: [ 2.55147123]
epoch-0; D_loss: [ 0.10351663]; G_loss: [ 2.6121099]
epoch-0; D_loss: [ 0.0957012]; G_loss: [ 2.71023536]
epoch-0; D_loss: [ 0.09293014]; G_loss: [ 2.78664756]
epoch-0; D_loss: [ 0.09101813]; G_loss: [ 2.87965035]
epoch-0; D_loss: [ 0.07222112]; G_loss: [ 2.98318458]
epoch-0; D_loss: [ 0.0691964]; G_loss: [ 3.07582593]
epoch-0; D_loss: [ 0.06445615]; G_l

...

epoch-3; D_loss: [ 0.00045306]; G_loss: [ 7.91097736]
epoch-3; D_loss: [ 0.00043497]; G_loss: [ 7.96069431]
epoch-3; D_loss: [ 0.00049028]; G_loss: [ 7.95203161]
epoch-3; D_loss: [ 0.00052244]; G_loss: [ 8.02214527]
epoch-3; D_loss: [ 0.0009166]; G_loss: [ 8.0004425]
epoch-3; D_loss: [ 0.00057886]; G_loss: [ 8.05801105]
epoch-3; D_loss: [ 0.00054766]; G_loss: [ 8.04842281]
epoch-3; D_loss: [ 0.0004029]; G_loss: [ 8.08086014]
epoch-3; D_loss: [ 0.00035744]; G_loss: [ 8.12941837]
epoch-3; D_loss: [ 0.00037293]; G_loss: [ 8.12363338]
epoch-3; D_loss: [ 0.0004141]; G_loss: [ 8.17512321]
epoch-3; D_loss: [ 0.00046036]; G_loss: [ 8.20204258]
epoch-3; D_loss: [ 0.00038423]; G_loss: [ 8.22887421]
epoch-3; D_loss: [ 0.00037789]; G_loss: [ 8.2595787]
epoch-3; D_loss: [ 0.00130426]; G_loss: [ 8.28219891]
epoch-3; D_loss: [ 0.00179813]; G_loss: [ 6.69181633]
epoch-3; D_loss: [ 0.00206977]; G_loss: [ 6.39983368]
epoch-3; D_loss: [ 0.00378166]; G_loss: [ 6.86476183]
epoch-3; D_loss: [ 0.00136574]; G

epoch-4; D_loss: [ 0.00011684]; G_loss: [ 9.28127193]
epoch-4; D_loss: [ 0.00014788]; G_loss: [ 9.25449181]
epoch-4; D_loss: [ 0.00011342]; G_loss: [ 9.29047585]
epoch-4; D_loss: [ 0.0001098]; G_loss: [ 9.35298443]
epoch-4; D_loss: [ 0.00010424]; G_loss: [ 9.34908199]
epoch-4; D_loss: [ 0.00010548]; G_loss: [ 9.41479015]
epoch-4; D_loss: [ 0.00010273]; G_loss: [ 9.45749569]
epoch-4; D_loss: [ 0.00015374]; G_loss: [ 9.5206871]
epoch-4; D_loss: [  8.70486765e-05]; G_loss: [ 9.54323959]
epoch-4; D_loss: [  8.56788101e-05]; G_loss: [ 9.60566711]
epoch-4; D_loss: [  7.70212355e-05]; G_loss: [ 9.64378166]
epoch-4; D_loss: [  8.06689859e-05]; G_loss: [ 9.6686573]
epoch-4; D_loss: [ 0.00010254]; G_loss: [ 9.67936707]
epoch-4; D_loss: [  7.61697593e-05]; G_loss: [ 9.71302509]
epoch-4; D_loss: [  8.70746881e-05]; G_loss: [ 9.74044514]
epoch-4; D_loss: [  7.70184197e-05]; G_loss: [ 9.78541088]
epoch-4; D_loss: [  6.07450493e-05]; G_loss: [ 10.059515]
epoch-4; D_loss: [  3.27284943e-05]; G_loss: [

epoch-4; D_loss: [  8.96469210e-05]; G_loss: [ 10.65182114]
epoch-4; D_loss: [  9.85991064e-05]; G_loss: [ 10.63534451]
epoch-4; D_loss: [ 0.00014228]; G_loss: [ 10.61205959]
epoch-4; D_loss: [ 0.00015631]; G_loss: [ 10.54713154]
epoch-4; D_loss: [  8.91428353e-05]; G_loss: [ 10.55783653]
epoch-4; D_loss: [  8.55437247e-05]; G_loss: [ 10.40586281]
epoch-4; D_loss: [ 0.00010291]; G_loss: [ 10.46725655]
epoch-4; D_loss: [ 0.00022751]; G_loss: [ 10.32941437]
epoch-4; D_loss: [ 0.00016011]; G_loss: [ 10.44710636]
epoch-4; D_loss: [ 0.00023642]; G_loss: [ 10.33443832]
epoch-4; D_loss: [ 0.0001294]; G_loss: [ 10.34549046]
epoch-4; D_loss: [ 0.00014087]; G_loss: [ 10.39947224]
epoch-4; D_loss: [  6.32744777e-05]; G_loss: [ 10.25451469]
epoch-4; D_loss: [  9.16698773e-05]; G_loss: [ 10.27488422]
epoch-4; D_loss: [ 0.00010069]; G_loss: [ 10.29672432]
epoch-4; D_loss: [ 0.00016857]; G_loss: [ 10.30045414]
epoch-4; D_loss: [ 0.00010265]; G_loss: [ 10.18536091]
epoch-4; D_loss: [ 0.00098841]; G_lo

epoch-4; D_loss: [ 0.00020763]; G_loss: [ 10.79029655]
epoch-4; D_loss: [ 0.00138924]; G_loss: [ 10.8418169]
epoch-4; D_loss: [ 0.00034871]; G_loss: [ 10.88344288]
epoch-4; D_loss: [ 0.00335329]; G_loss: [ 10.7169323]
epoch-4; D_loss: [ 0.0033862]; G_loss: [ 10.61138535]
epoch-4; D_loss: [ 0.0001611]; G_loss: [ 10.4040966]
epoch-4; D_loss: [ 0.0001751]; G_loss: [ 10.3466568]
epoch-4; D_loss: [ 0.00027042]; G_loss: [ 10.39068794]
epoch-4; D_loss: [ 0.00011881]; G_loss: [ 10.34176445]
epoch-4; D_loss: [ 0.00060881]; G_loss: [ 10.25957298]
epoch-4; D_loss: [ 0.00020019]; G_loss: [ 10.2946949]
epoch-4; D_loss: [ 0.00013768]; G_loss: [ 10.2846632]
epoch-4; D_loss: [ 0.00026247]; G_loss: [ 10.35204506]
epoch-4; D_loss: [ 0.00018602]; G_loss: [ 10.37083054]
epoch-4; D_loss: [ 0.00031535]; G_loss: [ 10.47144127]
epoch-4; D_loss: [ 0.00012733]; G_loss: [ 10.52967262]
epoch-4; D_loss: [ 0.00011914]; G_loss: [ 10.5512085]
epoch-4; D_loss: [ 0.00026991]; G_loss: [ 10.65831661]
epoch-4; D_loss: [  

epoch-4; D_loss: [ 0.00012798]; G_loss: [ 11.88307476]
epoch-4; D_loss: [  3.13654600e-05]; G_loss: [ 11.8815279]
epoch-4; D_loss: [  6.68905195e-05]; G_loss: [ 11.85743809]
epoch-4; D_loss: [  7.25255377e-05]; G_loss: [ 11.85079288]
epoch-4; D_loss: [  4.32745437e-05]; G_loss: [ 11.72759056]
epoch-4; D_loss: [  5.57053208e-05]; G_loss: [ 11.66903591]
epoch-4; D_loss: [  2.46878044e-05]; G_loss: [ 11.68759537]
epoch-4; D_loss: [ 0.00025839]; G_loss: [ 11.66835117]
epoch-4; D_loss: [  5.66840463e-05]; G_loss: [ 11.54837704]
epoch-4; D_loss: [  3.03748384e-05]; G_loss: [ 11.50520039]
epoch-4; D_loss: [  5.65250448e-05]; G_loss: [ 11.44198799]
epoch-4; D_loss: [ 0.00014691]; G_loss: [ 11.44985294]
epoch-4; D_loss: [  3.51391718e-05]; G_loss: [ 11.43318367]
epoch-4; D_loss: [  3.22734013e-05]; G_loss: [ 11.35660362]
epoch-4; D_loss: [  3.14123718e-05]; G_loss: [ 11.37591362]
epoch-4; D_loss: [  3.81952777e-05]; G_loss: [ 11.41164207]
epoch-4; D_loss: [ 0.00019605]; G_loss: [ 11.45885563]
e

In [41]:
df_SF=pd.DataFrame()
samples_per_class = 5000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [42]:
c_df.shape

(5000, 14)

In [43]:
df_SF.shape[0]

70000

In [44]:
c_gen = df_SF.values

In [45]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [46]:
z_gen.data.shape

torch.Size([70000, 100])

In [47]:
c_gen = Variable(torch.from_numpy(c_gen))

In [48]:
samples = G(z_gen, c_gen)

In [49]:
samples.data.shape

torch.Size([70000, 500])

In [50]:
#Taking the generated iVectors we will try to check the acc by MLP

In [51]:
train_X1 = samples.data.numpy()

In [52]:
type(train_X1)

numpy.ndarray

In [53]:
train_y1 = c_gen.data.numpy()

In [54]:
type(train_y1)

numpy.ndarray

In [55]:
train_y1

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]], dtype=float32)

In [56]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [57]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [58]:
#train_y

In [59]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [60]:
train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [61]:
train_X = train_X.values
train_y = train_y.values

In [62]:
train_y

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [63]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                7182      
__________

In [65]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [66]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [68]:
batch_size = 256
nb_epoch=20

In [ ]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

In [ ]:
#Frame label accuracy
score = model.evaluate(X_val, Y_val, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])